In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/file/d/1_NrNTXF4exPwcrLVz0k73NGlyizs5YUd/view?usp=sharing'

In [ ]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Survey_Results_Feb.csv') 
 
df = pd.read_csv('Survey_Results_Feb.csv')
print(df)

                                            StartDate  \
0                                          Start Date   
1   {"ImportId":"startDate","timeZone":"America/De...   
2                                 2022-04-24 09:51:53   
3                                 2022-04-24 10:37:00   
4                                 2022-04-25 01:42:23   
5                                 2022-04-25 02:00:57   
6                                 2022-04-25 05:32:43   
7                                 2022-04-26 06:08:38   
8                                 2022-04-25 02:21:26   
9                                 2022-04-21 06:07:43   
10                                2022-04-28 08:41:11   
11                                2022-04-30 03:19:37   
12                                2022-04-30 03:44:03   
13                                2022-04-30 08:10:14   
14                                2022-05-01 07:14:27   
15                                2022-04-24 10:32:08   
16                             

In [ ]:
df['Q100']

0     "@Boyanbc The Verses of the Disjointed Letters...
1                                 {"ImportId":"QID222"}
2                                                   NaN
3                                                     2
4                                                     2
5                                                     2
6                                                     2
7                                                     2
8                                                     2
9                                                   NaN
10                                                    2
11                                                    2
12                                                    2
13                                                    2
14                                                    2
15                                                  NaN
16                                                  NaN
17                                              

In [ ]:
#remove missing values (empty survey responses)
df = df.dropna(subset=['Q100'])

In [ ]:
#df['Q1']

results = df.loc[:, df.columns.str.startswith('Q')]

In [ ]:
results = results.drop([1])

In [ ]:
#check whether control tweets were answered correctly, row 3 and 14 are incorrect on second control tweet, so their responses will be deleted. 
print(results.to_markdown())

|    | Q1                                                                                                                                                                                                                            | Q2                                                                                                                                                                                 | Q3                                                                                                                               | Q4                                                                                                                                                                                                                                                       | Q5                                                                                                                                                       | Q6                                                 

In [ ]:
#remove rows 3 and 14 due to wrong control tweet answers
results = results.drop([3])
results = results.drop([14])

In [ ]:
#Final check if all control variables are answered correctly
print(results.to_markdown())

|    | Q1                                                                                                                                                                                                                            | Q2                                                                                                                                                                                 | Q3                                                                                                                               | Q4                                                                                                                                                                                                                                                       | Q5                                                                                                                                                       | Q6                                                 

In [ ]:
#switch rows as columns and columns as rows for fleiss kappa formula
results_transposed = results.transpose

In [ ]:
results_transposed

<bound method DataFrame.transpose of                                                    Q1  \
0   "Human rights groups have raised alarm over wh...   
4                                                   2   
5                                                   3   
6                                                   2   
7                                                   2   
8                                                   3   
10                                                  2   
11                                                  2   
12                                                  3   
13                                                  2   
23                                                  2   

                                                   Q2  \
0   @KingJames You have been awfully and painfully...   
4                                                   3   
5                                                   3   
6                                                 

In [ ]:
results_num = results.drop([0])

In [ ]:
results_T = results_num.transpose()

In [ ]:
results_T

,4,5,6,7,8,10,11,12,13,23
Q1,2,3,2,2,3,2,2,3,2,2
Q2,3,3,3,3,3,3,3,3,3,3
Q3,3,2,3,3,2,3,3,3,3,2
Q4,1,2,2,2,1,3,1,3,2,2
Q5,2,1,2,2,3,2,1,2,2,2
...,...,...,...,...,...,...,...,...,...,...
Q96,3,1,3,3,3,1,1,1,3,3
Q97,3,3,3,3,2,2,3,3,2,3
Q98,3,3,3,3,3,2,3,3,3,3
Q99,3,3,1,2,1,3,1,3,1,3


In [ ]:
results_T.iloc[101].value_counts(normalize=True)['2']

1.0

In [ ]:
#Count the 1, 2, and 3s per row per question
results_P = (results_T.apply(lambda x: x.value_counts(normalize=True), axis=1)
         .fillna(0)
         .astype(float))

In [ ]:
results_P

,1,2,3
Q1,0.0,0.7,0.3
Q2,0.0,0.0,1.0
Q3,0.0,0.3,0.7
Q4,0.3,0.5,0.2
Q5,0.2,0.7,0.1
...,...,...,...
Q96,0.4,0.0,0.6
Q97,0.0,0.3,0.7
Q98,0.0,0.1,0.9
Q99,0.4,0.1,0.5


In [ ]:
conditions = [
    (results_P['1'] > results_P['2']) & (results_P['1'] > results_P['3']),
    (results_P['2'] > results_P['1']) & (results_P['2'] > results_P['3']),
    (results_P['3'] > results_P['1']) & (results_P['3'] > results_P['2']),
    (results_P['1'] == results_P['2']),
    (results_P['2'] == results_P['3']),
    (results_P['1'] == results_P['3'])
]

choices = ['positive','neutral','negative', 'undecided', 'undecided', 'undecided']
results_P['Label'] = np.select(conditions, choices, default=np.nan)

In [ ]:
results_P['Label'].value_counts()

negative     46
neutral      43
positive     10
undecided     3
Name: Label, dtype: int64

In [ ]:
results_P.loc[results_P['Label']=='undecided']

,1,2,3,Label
Q8,0.0,0.5,0.5,undecided
Q26,0.5,0.5,0.0,undecided
Q29,0.0,0.5,0.5,undecided


In [ ]:
#Change undecided labels to either pos, neg, or neutral by use of an additional annotator
results_P.at['Q8','Label'] = 'neutral'
results_P.at['Q26','Label'] = 'positive'
results_P.at['Q29','Label'] = 'negative'

In [ ]:
results_P.loc[results_P['Label']=='undecided']

,1,2,3,Label


In [ ]:
results_P

,1,2,3,Label
Q1,0.0,0.7,0.3,neutral
Q2,0.0,0.0,1.0,negative
Q3,0.0,0.3,0.7,negative
Q4,0.3,0.5,0.2,neutral
Q5,0.2,0.7,0.1,neutral
...,...,...,...,...
Q96,0.4,0.0,0.6,negative
Q97,0.0,0.3,0.7,negative
Q98,0.0,0.1,0.9,negative
Q99,0.4,0.1,0.5,negative


In [ ]:
results['Q1'][0]

'"Human rights groups have raised alarm over what they describe as increasing censorship campaigns, following Russia\'s invasion of Ukraine. It\'s claimed Moscow has threatened independent media outlets reporting on the war.\n\nWATCH: https://t.co/69zHn5fiPU\n\n#Newzroom405 https://t.co/YeYs6WvKqf"'

In [ ]:
results

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,...,Q91,Q92,Q93,Q94,Q95,Q96,Q97,Q98,Q99,Q100
0,"""Human rights groups have raised alarm over wh...",@KingJames You have been awfully and painfully...,"""@PrillLee @MarthaP29033882 @DanRather It is n...",@AntoineSallesP @JoshuaHabib4 @JLMelenchon Why...,"@IAmMrKobayashi World War III; Full video, How...",@BurggrabenH #sauce?? - with friends like Chin...,@johnpavlovitz No doubt he would be ordering U...,@POTUS Joe Biden did that 👉 Dow futures tumble...,"""Hats off Ukraine. 🥺\n#Ukraine #Russia #Volody...","""#IStandWithUkraine\n\nUkraine's ambassador to...",...,"""'#Putin will not hesitate to send agents to k...","""@ua_parliament @NATO @UN Where is your help, ...","""@OttawaCitizen Appearing for the last time in...","""Additional UK forces have started to arrive i...",CIO Larry Adam and other key Raymond James str...,There are 7 or 8 Billionaires in the UKRAINE! ...,"""@EP_President @Europarl_EN @EU_Commission STO...","""@vonderleyen STOP RUSSIAN AGGRESSION AGAINST ...","""""However difficult it may be, our pain will b...","""@Boyanbc The Verses of the Disjointed Letters..."
4,2,3,3,1,2,1,2,3,3,3,...,2,3,3,2,2,3,3,3,3,2
5,3,3,2,2,1,3,2,3,1,1,...,3,3,3,2,1,1,3,3,3,2
6,2,3,3,2,2,3,2,2,1,2,...,2,3,3,2,2,3,3,3,1,2
7,2,3,3,2,2,3,2,2,3,2,...,2,3,1,2,2,3,3,3,2,2
8,3,3,2,1,3,2,3,2,2,3,...,3,1,3,2,1,3,2,3,1,2
10,2,3,3,3,2,3,3,3,1,2,...,2,3,3,2,2,1,2,2,3,2
11,2,3,3,1,1,1,1,2,1,3,...,2,3,1,1,2,1,3,3,1,2
12,3,3,3,3,2,3,3,3,1,1,...,3,3,3,1,2,1,3,3,3,2
13,2,3,3,2,2,3,2,2,1,2,...,2,2,2,2,2,3,2,3,1,2


In [ ]:
results_P['text'] = results.T[0]

In [ ]:
results_P = results_P[['text','1','2','3','Label']]

In [ ]:
results_P['Label'].value_counts()

negative    47
neutral     44
positive    11
Name: Label, dtype: int64

In [ ]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
results_P.to_csv('Survey_Results_Labeled_Feb.csv')
!cp Survey_Results_Labeled_Feb.csv "/content/drive/My Drive/Colab Notebooks"

In [ ]:
#DONT NEED THIS 
#Count the 1, 2, and 3s per row per question
#df1 = (results_T.apply(lambda x: x.value_counts(), axis=1)
         #.fillna(0)
         #.astype(int))

In [ ]:
#from statsmodels.stats.inter_rater import fleiss_kappa
#fleiss_kappa(df1,method='fleiss')

0.2583195210117539

In [ ]:
df2 = results.drop([0])

In [ ]:
!pip install krippendorff
import krippendorff

In [ ]:

df3 = df2.values.tolist()

In [ ]:
df4 = [[int(float(j)) for j in i] for i in df3]

In [ ]:
res = krippendorff.alpha(df4)

In [ ]:
res

0.4309567146735934